In [11]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt

import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.optim as optim
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence

from torch.utils.data import DataLoader
import glob



#### Actual Data

In [19]:
## generate test / train split
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, seq_len, batch_size):
        self.x, self.y = x_set, y_set
        self.seq_len = seq_len
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.seq_len)))

    def __getitem__(self, idx):
        start_idx = idx * self.seq_len
        end_idx = (idx + 1) * self.seq_len

        batch_x = self.x[start_idx:end_idx]
        batch_y = self.y[start_idx:end_idx]

        batch_x = torch.from_numpy(batch_x)
        batch_y = torch.from_numpy(batch_y)
        
        # Pad sequences to ensure they have the same length within the batch
        pad_len = self.seq_len - batch_x.shape[0]
        if pad_len > 0:
            pad_shape = (pad_len,) + batch_x.shape[1:]
            pad_shape_y = (pad_len,) + batch_y.shape[1:]
            
            batch_x = torch.cat([batch_x, torch.zeros(pad_shape)], dim=0)
            batch_y = torch.cat([batch_y, torch.zeros(pad_shape_y)], dim=0)

        return batch_x, batch_y

    def on_epoch_end(self):
        indices = np.arange(len(self.x))
        np.random.shuffle(indices)
        self.x = self.x[indices]
        self.y = self.y[indices]
        
def generate_data_split(subject_id):
    # Get CSV files list from a folder
    train_path = './Train'
    test_path = './Test'
    
    csv_path = './ProcessedDatasets/Knot_Tying'
    
    csv_files = glob.glob(csv_path + "/*.csv")
    
    train_df_list = []
    test_df_list = []
    
    for file in csv_files:
        if(subject_id in file):
            test_df_list.append(pd.read_csv(file))
#             print(file)
        else:
            train_df_list.append(pd.read_csv(file))
            

    print('Train Subject Trials: ',len(train_df_list))
    print('Test Subject Trials: ',len(test_df_list))
    
    # Concatenate all DataFrames
    train_df   = pd.concat(train_df_list, ignore_index=True)
    test_df   = pd.concat(test_df_list, ignore_index=True)

    
    lb = preprocessing.LabelBinarizer()

    train_labels= train_df.pop('label')
    train_features = train_df

    test_labels= test_df.pop('label')
    test_features = test_df


    all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
    lb.fit(all_class_names)

    train_labels = lb.transform(train_labels)
    test_labels = lb.transform(test_labels)
    
    train_x = train_features.to_numpy()
    train_y = train_labels

    test_x = test_features.to_numpy()
    test_y = test_labels

    seq_len = 30
    batch_size = 64

    train_dataset = CustomDataset(train_x, train_y, seq_len, batch_size)
    test_dataset = CustomDataset(test_x, test_y, seq_len, batch_size)

    train_dataloader = tf.data.Dataset.from_generator(
        lambda: train_dataset,
        output_signature=(
            tf.TensorSpec(shape=( seq_len, train_x.shape[1]), dtype=tf.float32),
            tf.TensorSpec(shape=( seq_len, train_y.shape[1]), dtype=tf.float32),
        ),
    # )
    ).repeat()
    train_dataloader = train_dataloader.batch(batch_size)



    test_dataloader = tf.data.Dataset.from_generator(
        lambda: test_dataset,
        output_signature=(
            tf.TensorSpec(shape=( seq_len, test_x.shape[1]), dtype=tf.float32),
            tf.TensorSpec(shape=( seq_len, test_y.shape[1]), dtype=tf.float32),
        ),
    )
    test_dataloader = test_dataloader.batch(batch_size)
    
    return train_dataloader, test_dataloader
    
# subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']

# for subject in subjects:
#     generate_data_split(subject)

In [26]:


# # Get CSV files list from a folder
# train_path = './Train'
# test_path = './Test'
# train_csv_files = glob.glob(train_path + "/*.csv")
# test_csv_files = glob.glob(test_path + "/*.csv")

# # Read each CSV file into DataFrame
# # This creates a list of dataframes
# train_df_list = (pd.read_csv(file) for file in train_csv_files)
# test_df_list = (pd.read_csv(file) for file in test_csv_files)

# # Concatenate all DataFrames
# train_df   = pd.concat(train_df_list, ignore_index=True)
# test_df   = pd.concat(test_df_list, ignore_index=True)

# lb = preprocessing.LabelBinarizer()

# train_labels= train_df.pop('label')
# train_features = train_df

# test_labels= test_df.pop('label')
# test_features = test_df


# all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
# lb.fit(all_class_names)

# train_labels = lb.transform(train_labels)
# test_labels = lb.transform(test_labels)


# # seq_len = 30
# # batch_size = 30

# # class DataGenerator(Sequence):
# #     def __init__(self, x_set, y_set, batch_size):
# #         self.x, self.y = x_set, y_set
# #         self.batch_size = batch_size

# #     def __len__(self):
# #         return int(np.ceil(len(self.x) / float(self.batch_size)))

# #     def __getitem__(self, idx):
# #         batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
# #         batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

# #         return batch_x, batch_y

# # train_gen = DataGenerator(train_features, train_labels, batch_size)
# # test_gen = DataGenerator(test_features, test_labels, batch_size)


In [27]:
# class CustomDataset(Dataset):
#     def __init__(self, x_set, y_set, seq_len):
#         self.x, self.y = x_set, y_set
#         self.seq_len = seq_len
        
#     def __len__(self):
#         return int(np.ceil(len(self.x) / float(self.seq_len)))

#     def __getitem__(self, idx):
#         batch_x = self.x[idx * self.seq_len:(idx + 1) * self.seq_len]
#         batch_y = self.y[idx * self.seq_len:(idx + 1) * self.seq_len]
#         batch_x = torch.from_numpy(batch_x)
#         batch_y = torch.from_numpy(batch_y)
#         return batch_x, batch_y
    
# train_x = train_features.to_numpy()
# train_y = train_labels
    
# test_x = test_features.to_numpy()
# test_y = test_labels

# seq_len = 30
# batch_size = 1


# train_dataset = CustomDataset(train_x, train_y, seq_len)
# test_dataset = CustomDataset(test_x, test_y, seq_len)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)




In [28]:
# import numpy as np
# import tensorflow as tf

# class CustomDataset(tf.keras.utils.Sequence):
#     def __init__(self, x_set, y_set, seq_len, batch_size):
#         self.x, self.y = x_set, y_set
#         self.seq_len = seq_len
#         self.batch_size = batch_size

#     def __len__(self):
#         return int(np.ceil(len(self.x) / float(self.seq_len)))

#     def __getitem__(self, idx):
#         start_idx = idx * self.seq_len
#         end_idx = (idx + 1) * self.seq_len

#         batch_x = self.x[start_idx:end_idx]
#         batch_y = self.y[start_idx:end_idx]

#         batch_x = torch.from_numpy(batch_x)
#         batch_y = torch.from_numpy(batch_y)
        
#         # Pad sequences to ensure they have the same length within the batch
#         pad_len = self.seq_len - batch_x.shape[0]
#         if pad_len > 0:
#             pad_shape = (pad_len,) + batch_x.shape[1:]
#             pad_shape_y = (pad_len,) + batch_y.shape[1:]
            
#             batch_x = torch.cat([batch_x, torch.zeros(pad_shape)], dim=0)
#             batch_y = torch.cat([batch_y, torch.zeros(pad_shape_y)], dim=0)

#         return batch_x, batch_y

#     def on_epoch_end(self):
#         indices = np.arange(len(self.x))
#         np.random.shuffle(indices)
#         self.x = self.x[indices]
#         self.y = self.y[indices]

# train_x = train_features.to_numpy()
# train_y = train_labels

# test_x = test_features.to_numpy()
# test_y = test_labels

# seq_len = 30
# batch_size = 64

# train_dataset = CustomDataset(train_x, train_y, seq_len, batch_size)
# test_dataset = CustomDataset(test_x, test_y, seq_len, batch_size)

# train_dataloader = tf.data.Dataset.from_generator(
#     lambda: train_dataset,
#     output_signature=(
#         tf.TensorSpec(shape=( seq_len, train_x.shape[1]), dtype=tf.float32),
#         tf.TensorSpec(shape=( seq_len, train_y.shape[1]), dtype=tf.float32),
#     ),
# # )
# ).repeat()
# train_dataloader = train_dataloader.batch(batch_size)



# test_dataloader = tf.data.Dataset.from_generator(
#     lambda: test_dataset,
#     output_signature=(
#         tf.TensorSpec(shape=( seq_len, test_x.shape[1]), dtype=tf.float32),
#         tf.TensorSpec(shape=( seq_len, test_y.shape[1]), dtype=tf.float32),
#     ),
# )
# test_dataloader = test_dataloader.batch(batch_size)



In [29]:
# def generate_data_split(subject_id):
#     # Get CSV files list from a folder
#     train_path = './Train'
#     test_path = './Test'
    
#     csv_path = './Dataset'
    
#     csv_files = glob.glob(csv_path + "/*.csv")
    
#     train_df_list = []
#     test_df_list = []
    
#     for file in csv_files:
#         if(subject_id in file):
#             test_df_list.append(pd.read_csv(file))
# #             print(file)
#         else:
#             train_df_list.append(pd.read_csv(file))
            

#     print('Train Subject Trials: ',len(train_df_list))
#     print('Test Subject Trials: ',len(test_df_list))
    
#     # Concatenate all DataFrames
#     train_df   = pd.concat(train_df_list, ignore_index=True)
#     test_df   = pd.concat(test_df_list, ignore_index=True)

    
#     lb = preprocessing.LabelBinarizer()

#     train_labels= train_df.pop('label')
#     train_features = train_df

#     test_labels= test_df.pop('label')
#     test_features = test_df


#     all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
#     lb.fit(all_class_names)

#     train_labels = lb.transform(train_labels)
#     test_labels = lb.transform(test_labels)
    
#     train_x = train_features.to_numpy()
#     train_y = train_labels

#     test_x = test_features.to_numpy()
#     test_y = test_labels

#     seq_len = 30
#     batch_size = 64

#     valid_test_split = 0.8
#     # Step 2: Split the remaining data into validation and test sets
#     val_x, test_x, val_y, test_y = train_test_split(
#     test_x, test_y, test_size=valid_test_split, random_state=42)
    
#     train_dataset = CustomDataset(train_x, train_y, seq_len, batch_size)
    
#     val_dataset = CustomDataset(val_x, val_y, seq_len, batch_size)
    
#     test_dataset = CustomDataset(test_x, test_y, seq_len, batch_size)

#     train_dataloader = tf.data.Dataset.from_generator(
#         lambda: train_dataset,
#         output_signature=(
#             tf.TensorSpec(shape=( seq_len, train_x.shape[1]), dtype=tf.float32),
#             tf.TensorSpec(shape=( seq_len, train_y.shape[1]), dtype=tf.float32),
#         ),
#     # )
#     ).repeat()
#     train_dataloader = train_dataloader.batch(batch_size)


#     val_dataloader = tf.data.Dataset.from_generator(
#         lambda: val_dataset,
#         output_signature=(
#             tf.TensorSpec(shape=( seq_len, test_x.shape[1]), dtype=tf.float32),
#             tf.TensorSpec(shape=( seq_len, test_y.shape[1]), dtype=tf.float32),
#         ),
#     )
#     val_dataloader = val_dataloader.batch(batch_size)
    

#     test_dataloader = tf.data.Dataset.from_generator(
#         lambda: test_dataset,
#         output_signature=(
#             tf.TensorSpec(shape=( seq_len, test_x.shape[1]), dtype=tf.float32),
#             tf.TensorSpec(shape=( seq_len, test_y.shape[1]), dtype=tf.float32),
#         ),
#     )
#     test_dataloader = test_dataloader.batch(batch_size)
    
#     return train_dataloader, val_dataloader, test_dataloader

In [30]:
# for item in train_dataloader:
#     print(item[0].shape)
#     break

In [31]:
# row = 41 # -2 ## 43 -> G13 i.e 41 

# print(lb.classes_)

# print(lb.inverse_transform(test_labels)[row])
# print(test_labels[row])

# # print(train_labels.iloc[row])
# # print(np.argmax(test_labels[row]))

In [32]:
# from utils import *


# all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']

# train_loader, test_loader = get_dataloaders(task="Knot_Tying", subject_id_to_exclude=8, observation_window=30,prediction_window=30, batch_size=64, one_hot=True, class_names = all_class_names)



In [14]:
def positional_encoding(length, depth):
    depth = depth/2

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

    angle_rates = 1 / (10000**depths)         # (1, depth)
    angle_rads = positions * angle_rates      # (pos, depth)

    pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

In [15]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, input_size, d_model):
        super().__init__()
        self.d_model = d_model
        # self.embedding = tf.keras.layers.Embedding(input_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(length=2048, depth=d_model)
        self.linear = tf.keras.layers.Dense(d_model)

    def call(self, x):
        x = self.linear(x)
        length = tf.shape(x)[1]

        # This factor sets the relative scale of the embedding and positonal_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

### Encoder

In [16]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    

        
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


### Model Build

In [17]:
def build_model(
    input_shape,
    output_dim,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    x = PositionalEmbedding(input_size=inputs.shape[1], d_model=head_size)(x)
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

#     print("bf_glbl",x.shape)
    x = layers.GlobalAveragePooling1D(data_format="channels_first", keepdims=True)(x)
#     print("af_glbl",x.shape)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    print(output_dim)
    outputs = layers.Dense(output_dim, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [24]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

2023-07-28 12:22:59.973853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 12:22:59.974001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 12:22:59.974080: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 12:22:59.974208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-28 12:22:59.974311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

### Train and Evaluate

In [26]:
# input_shape = (30,66)
# input_shape = train_features.shape

# train_features = train_features.to_numpy()
# train_features = train_features.reshape((train_features.shape[0], train_features.shape[1], 1))
# input_shape = train_features.shape
subject = "S04"
train_dataloader, test_dataloader = generate_data_split(subject)
train_features, train_labels = next(iter(train_dataloader))

input_shape = train_features.shape[1:]
output_dim = train_labels.shape[2]

print("input_dim: ",input_shape)
print("output_dim: ",output_dim)



model = build_model(
    input_shape,
    output_dim,
    head_size=32,
    num_heads=2,
    ff_dim=64,
    num_transformer_blocks=2,
    mlp_units=[256,128],
    mlp_dropout=0.2,
    dropout=0.2,
)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["categorical_accuracy"],
)
# # # # model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

epochs = 100
# steps_per_epoch = train_dataloader.__len__()//1
# print("steps_per", steps_per_epoch)


subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']


model.save_weights('./checkpoints/blank_state')

for subject in subjects:
    
    print(" *** --------START TRAIN--------- *** ")
    print("SUBJECT EXCLUDED: ",subject)
    
    train_dataloader, test_dataloader = generate_data_split(subject)

    model.load_weights('./checkpoints/blank_state')

    model.fit(
        train_dataloader,
        epochs=epochs,
        callbacks=callbacks,
        steps_per_epoch=500,
#         validation_data = valid_dataloader
    )

    print(" *** -------END TRAIN-------- *** ")
    print(" *** -------START EVAL-------- *** ")
    
    model.evaluate(test_dataloader, verbose=1)

    print(" *** -------END EVAL-------- *** ")
    



## Pytorch dataloader implementation

# def gen(torch_loader):
#     for x,y in torch_loader:
#         yield (x,y)

# train = gen(train_dataloader)

# epochs = 300
# steps_per_epoch = train_dataloader.__len__()//5
# print("steps_per", steps_per_epoch)

# model.fit(
#     train,
#     epochs=epochs,
#     callbacks=callbacks,
#     steps_per_epoch=steps_per_epoch
# )

# # model.evaluate(x_test, y_test, verbose=1)


Train Subject Trials:  31
Test Subject Trials:  5
input_dim:  (30, 66)
output_dim:  14


2023-07-28 12:32:21.252823: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.0KiB (rounded to 524288)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-28 12:32:21.252889: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-07-28 12:32:21.252906: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 31, Chunks in use: 31. 7.8KiB allocated for chunks. 7.8KiB in use in bin. 245B client-requested in use in bin.
2023-07-28 12:32:21.252921: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.0KiB allocated for chunks. 1.0KiB in use in bin. 1.0KiB client-requested in use in bin.
2023-07-28 12:32:21.252932: I tensorfl

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [8]:
subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']

for subject in subjects:
    
    print(" *** --------START--------- *** ")
    print("SUBJECT EXCLUDED: ",subject)
    
    train_dataloader, test_dataloader = generate_data_split(subject)
    model.evaluate(test_dataloader, verbose=1)
    

    print(" *** --------END--------- *** ")




 *** --------START--------- *** 
SUBJECT EXCLUDED:  S02
Train Subject Trials:  32
Test Subject Trials:  4
4/4 [==============================] - 0s 14ms/step - loss: 0.0771 - categorical_accuracy: 0.9830
 *** --------END--------- *** 
 *** --------START--------- *** 
SUBJECT EXCLUDED:  S03
Train Subject Trials:  31
Test Subject Trials:  5
3/3 [==============================] - 0s 19ms/step - loss: 0.0755 - categorical_accuracy: 0.9796
 *** --------END--------- *** 
 *** --------START--------- *** 
SUBJECT EXCLUDED:  S04
Train Subject Trials:  31
Test Subject Trials:  5
3/3 [==============================] - 0s 19ms/step - loss: 0.0591 - categorical_accuracy: 0.9806
 *** --------END--------- *** 
 *** --------START--------- *** 
SUBJECT EXCLUDED:  S05
Train Subject Trials:  31
Test Subject Trials:  5
4/4 [==============================] - 0s 16ms/step - loss: 0.0815 - categorical_accuracy: 0.9772
 *** --------END--------- *** 
 *** --------START--------- *** 
SUBJECT EXCLUDED:  S06
Trai

In [ ]:
# !pip install -q -U keras-tuner


In [ ]:
# import keras_tuner as kt

# train_features, train_labels = next(iter(train_dataloader))

# input_shape = train_features.shape[1:]
# output_dim = train_labels.shape[2]

# print("input_dim: ",input_shape)
# print("output_dim: ",output_dim)


# def model_builder(hp):
    
    
#     hp_headsize = hp.Int('units', min_value=16, max_value=512, step=16)
#     hp_numheads = hp.Int('units', min_value=2, max_value=16, step=2)
#     hp_ff_dim = hp.Int('units', min_value=1, max_value=512, step=16)
#     hp_num_transformer_blocks = hp.Int('units', min_value=1, max_value=16, step=2)

#     model = build_model(
#     input_shape,
#     output_dim,
#     head_size=hp_headsize,
#     num_heads=hp_numheads,
#     ff_dim=hp_ff_dim,
#     num_transformer_blocks=hp_num_transformer_blocks,
#     mlp_units=[128],
#     mlp_dropout=0.4,
#     dropout=0.25,
#     )

#     model.compile(
#         loss="categorical_crossentropy",
#         optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#         metrics=["categorical_accuracy"],
#     )
    
#     return model


# tuner = kt.Hyperband(model_builder,
#                      objective='val_accuracy',
#                      max_epochs=10,
#                      factor=3,
#                      project_name='intro_to_kt')


In [ ]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
# tuner.search(train_dataloader, epochs=10, steps_per_epoch=500, callbacks=[stop_early])

# # Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)


In [ ]:
model.evaluate(test_dataloader, verbose=1)

# test_features, test_labels = next(iter(test_dataloader))

# y_pred = model(test_features)

# relu = nn.ReLU()


# for idx,y in enumerate(y_pred) :
#     for i,sample in enumerate(y):
#         print(np.argmax(test_labels[idx][i].numpy()))
#         print(np.argmax(sample))
#         print('-----')
        
#     break

In [7]:
model.save_weights('./checkpoints/progress_v4_07_27')

In [ ]:
# # Restore the weights
# model.load_weights('./checkpoints/progress_v2')
